In [1]:
import pandas as pd
from IPython.display import clear_output

In [2]:
import time
import random

In [3]:
df = pd.read_parquet('./4YR_cleaned_USAS.parquet')
df.head(2)

,Event,Conf_Rank,Time,Name,School,Standard,Date,Gender,Division,Season,SeasonYear,Time_Adj,Time_Sec,Points,Div_Rank,NCAA_Rank,Power
0,50 Freestyle SCY Male,2,20.20,Tiago ASAKAWA,UMBC,NS,2020-02-14,M,I,19_20,2020,0 days 00:00:20.200000,20.20,664.818090,252.0,291.0,15.96
1,50 Freestyle SCY Male,4,20.72,Niklas Weigelt,UMBC,NS,2020-02-14,M,I,19_20,2020,0 days 00:00:20.720000,20.72,616.009888,542.0,690.0,8.38


In [4]:
df = df.drop('Power',axis=1)
df.head(2)

,Event,Conf_Rank,Time,Name,School,Standard,Date,Gender,Division,Season,SeasonYear,Time_Adj,Time_Sec,Points,Div_Rank,NCAA_Rank
0,50 Freestyle SCY Male,2,20.20,Tiago ASAKAWA,UMBC,NS,2020-02-14,M,I,19_20,2020,0 days 00:00:20.200000,20.20,664.818090,252.0,291.0
1,50 Freestyle SCY Male,4,20.72,Niklas Weigelt,UMBC,NS,2020-02-14,M,I,19_20,2020,0 days 00:00:20.720000,20.72,616.009888,542.0,690.0


In [5]:
name_info = df[['Name', 'School']].drop_duplicates()
name_info.shape
# Add 'Power' column with all values set to -1
name_info['Power'] = -1.0
# Add 'Link' column with all values set to an empty string
name_info['Link'] = ''
name_info['Hometown'] = ''
# Check the result
name_info.head()

,Name,School,Power,Link,Hometown
0,Tiago ASAKAWA,UMBC,-1.0,,
1,Niklas Weigelt,UMBC,-1.0,,
2,Ben Beldner,Binghamton,-1.0,,
3,Justin Meyn,Binghamton,-1.0,,
4,Joshua Franco,NJIT,-1.0,,


In [6]:
import requests
import csv
from bs4 import BeautifulSoup as bs
import pandas as pd
import time as _time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import re

def getpi(swimmer_name):
    swimmer_name_url = 'https://swimcloud.com/recruiting/rankings/?name='+swimmer_name.replace(' ','+')
    name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
    name_url.encoding = 'utf-8'
    name_soup = bs(name_url.text, 'html.parser')
    # print(name_soup)
    # print(name_soup)
    # print(name_url.text)
    names=[]
    links=[]
    idxs=[]
    teams=[]
    hometowns=[]
    if (name_soup.find('tbody') == None):
        # print('No tbody')
        return None
    swimmer_list = name_soup.find('tbody').find_all('tr')
    i=0
    for swimmer in swimmer_list:
        if len(name_soup.find_all('td', {'class' : 'u-text-end'})) == 0:
            # print('No idx found')
            break
        name = swimmer.find(class_='u-text-semi')
        name_str = str(name)
        start = name_str.find('>') + 1
        end = name_str.find('</a>')
        # Extract the text between '>' and '<'
        name_raw = name_str[start:end]
        if (name_raw.lower() == swimmer_name.lower()):
            names.append(name_raw)
            swimmer_link = name['href'] if name else ''
            links.append(swimmer_link)
            # if (swimmer_name == 'Reese Samuel'):
            #     print(i)
            #     print(name_soup.find_all('td', {'class' : 'u-text-end'})[i])
            # idxs.append(float(name_soup.find_all('td', {'class' : 'u-text-end'})[i].text.strip()))
            team_td = swimmer.find_all('td', class_='u-text-center hidden-xs')[0]
            # Find the <img> tag within the <td>
            img_tag = team_td.find('img')
            if img_tag == None:
                teams.append('')
            else:
                # Extract the 'alt' attribute value, which contains the team name
                team_name = img_tag['alt'][:-5]
                teams.append(team_name)
    
    
            # swimmer_name_url = 'https://swimcloud.com'+swimmer_link
            # name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
            # name_url.encoding = 'utf-8'
            # name_soup2 = bs(name_url.text, 'html.parser')
            # if (name_soup2.find('ul', class_='o-list-inline o-list-inline--dotted') == None):
            #     return None
            # ul = name_soup2.find('ul', class_='o-list-inline o-list-inline--dotted')
            # location_li = ul.find_all('li')[0]
            
            # # Extract and print the location text
            # location = location_li.text.strip()
            # hometowns.append(location)
        i = i + 1
    # print(names)
    # print(links)
    # print(hometowns)
    return [names,links,idxs,teams,hometowns]


In [7]:
all_names = df.Name.unique()
j = 0
modified = 0

for j in range(len(all_names)):
    name = all_names[j]
    # print('about to parse ' + name)
    if (j > -1):
        lists = getpi(name)
        if (lists == None):
            print('No tbody')
            time.sleep(20)
            j = j - 1
        else:
            if len(lists[0]) == 1:
                # name_info.loc[(name_info.Name == name), 'Power'] = lists[2][0]
                name_info.loc[(name_info.Name == name), 'Link'] = lists[1][0]
                # name_info.loc[(name_info.Name == name), 'Hometown'] = lists[4][0]
                # print(" Power " + str(lists[2][0]))
                print('Single')
                modified += 1
            elif len(lists[0]) > 1:
                print(name)
                x_old = name_info[name_info.Name == name]['School'].value_counts()
                x_old = x_old[x_old > 0]
                
                for school in x_old.index:
                    k = 0
                    match = False
                    for school2 in lists[3]:
                        if (school in school2 or school2 in school and school2 != ''):
                            # name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Power'] = lists[2][k]
                            name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Link'] = lists[1][k]
                            # name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Hometown'] = lists[4][k]
                            modified += 1
                            match = True
                            # print('Name ' + str(name) + " School " + str(school) + " Power " + str(lists[2][k]))
                            print('Match')
                    if not match:
                        print('No Match')
                        x_old = name_info[name_info.Name == name]['School'].value_counts()
                        x_old = x_old[x_old > 0]
                        
                        for school in x_old.index:
                            k = 0
                            match = False
                            best_match = ''
                            highest_score = 0
                        
                            # Find the closest match using fuzzy string matching
                            for school2 in lists[3]:
                                if school2 != '':
                                    score = fuzz.ratio(school, school2)
                                    if score > highest_score:
                                        highest_score = score
                                        best_match = school2
                        
                            # Define a threshold for a "good" match
                            threshold = 80
                        
                            if highest_score >= threshold:
                                # Perform the match with the closest match
                                name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Power'] = lists[2][k]
                                name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Link'] = lists[1][k]
                                # name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Hometown'] = lists[4][k]
                                modified += 1
                                match = True
                                print('Match 2nd try with:', best_match, " with score = ", highest_score)
                        
                            if not match:
                                print('No Match AGAIN for:', school, " with score = ", highest_score)
                                name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Link'] = 'NoMatch'
                
                    k += 1
                        name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Link'] = 'NoMatch'

                        k += 1
            else:
                print('Empty')
                # df.loc[(df.Name == name), 'Power'] = -1

    j = j + 1
    # time.sleep(random.uniform(0, 1))
    if (j % 50 == 0):
        clear_output() 
    if (j % 10 == 0):
        print(j)
        print(modified)

IndentationError: unexpected indent (109286488.py, line 75)

In [ ]:
name_info.to_csv('name_info.csv',index=False)

In [ ]:
name_info.Link.value_counts()

In [ ]:
name_mapping = {
    'Fla Atlantic (M)': 'Florida Atlantic University',
    'Fla Atlantic (W)': 'Florida Atlantic University',
    'Loyola (MD)': 'Loyola University (Maryland)',
    'U.S. Navy': 'U.S. Naval Academy (Navy)',
    'UW-Stevens Point': 'University of Wisconsin-Stevens Point',
    'UWEC': 'University of Wisconsin-Eau Claire',
    'Wash U. MO': 'Washington University (Missouri)',
    'Georgia Tech': 'Georgia Institute of Technology',
    'Virginia MI': 'Virginia Military Institute',
    'Iona Coll': 'Iona University',
    'Wheaton IL': 'Wheaton College (Illinois)',
    'TCNJ': 'College of New Jersey, The',
    'UMSL': 'University of Missouri-St. Louis',
    'Henderson St.': 'Henderson State University',
    'IWU': 'Indiana Wesleyan University',
    'SMU': 'Southern Methodist University',
    'UCSB': 'University of California-Santa Barbara',
    'Missouri St (W)': 'Missouri State University',
    'NC State': 'North Carolina State University',
    'USMMA': 'U.S. Merchant Marine Academy',
    'NDC': 'Notre Dame (Ohio) College',
    'Air Force (W)': 'U.S. Air Force Academy',
    'St. Norbert': 'St Norbert College',
    'W&L': 'Washington and Lee University',
    'UNC Wilmington': 'University of North Carolina, Wilmington',
    'SHIP': 'Shippensburg University',
    'IU Indianapolis': 'Indiana University-Indianapolis',
    'Cal Poly': 'California Polytechnic State University',
    'BSC': 'Birmingham Southern College',
    'Wyoming (M)': 'University of Wyoming',
    'MIT': 'Massachusetts Institute of Technology',
    'Bryant U (W)': 'Bryant University',
    'St. Louis': 'Saint Louis University',
    'UN Omaha': 'University of Nebraska, Omaha',
    'TAMPA': 'University of Tampa',
    'Claremont MS': 'Claremont McKenna-Harvey Mudd-Scripps Colleges',
    'Wheaton MA': 'Wheaton College (Massachusetts)',
    'Evansville (M)': 'University of Evansville',
    'Fla Atlantic (M)': 'Florida Atlantic University',
    'UNE': 'University of New England',
    'Trinity C.': 'Trinity College (Connecticut)',
    'SUNY Geneseo': 'State University of New York at Geneseo',
    'Catholic UA': 'Catholic University',
    'Colorado St.': 'Colorado State University',
    'Frostburg St.': 'Frostburg State University',
    'UNC': 'University of North Carolina, Chapel Hill',
    'F&M': 'Franklin & Marshall College',
    'SUNY Brockport': 'State University of New York at Brockport',
    'Indy': 'University of Indianapolis',
    'Wyoming (W)': 'University of Wyoming',
    'NYU': 'New York University',
    'RIT': 'Rochester Institute of Technology'
}

In [ ]:
name_info['School'] = df['School'].replace(name_mapping)

In [ ]:
name_info[(name_info.Link=='NoMatch')].School.value_counts().head(50)

In [ ]:
name_info

In [ ]:
name_info.info()

In [ ]:
name_info.Link = name_info.Link.astype('string')
name_info.Hometown = name_info.Hometown.astype('string')

In [ ]:
name_info.to_parquet('name_info_dtypes.parquet',index=False)